In [40]:
import sqlite3
import pandas as pd
import numpy as np

#1 Kết nối đến SQLite (CSDL trong bộ nhớ)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng student
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Tạo bảng course
cursor.execute("""
CREATE TABLE courses (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Tạo DataFrame cho bảng student
student_data = {
    'student_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'name': ["Nguyen Minh Hoang", "Tran Thi Lan", "Pham Van Nam", "Le Thanh Huyen", "Vu Quoc Anh", "Dang Thuy Linh", "Bui Tien Dung", "Ho Ngoc Mai", "Duong Huu Phuc", "Cao Thi Hanh"],
    'class': ["May Tinh", "Kinh Te", "Toan Tin", "Toan Tin", "May Tinh", "May Tinh", "Kinh Te", "Toan Tin", "Kinh Te", "May Tinh"],
    'course_id': [12, 34, np.nan, 20, 24, 24, 34, 20, np.nan, np.nan],
    'score': [6.7, 9.2, 7.9, 7.2, 8.0, 5.5, 9.2, 8.8, 7.2, 7.0]
}
students = pd.DataFrame(student_data)

# Tạo DataFrame cho bảng course
course_data = {
    'id': [12, 34, 26, 20, 24],
    'course_name': ["Giai tich", "Thong ke", "Tin hoc", "Dai so", "Lap trinh"]
}
courses = pd.DataFrame(course_data)
print(students)
print(courses)


   student_id               name     class  course_id  score
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7
1           2       Tran Thi Lan   Kinh Te       34.0    9.2
2           3       Pham Van Nam  Toan Tin        NaN    7.9
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2
4           5        Vu Quoc Anh  May Tinh       24.0    8.0
5           6     Dang Thuy Linh  May Tinh       24.0    5.5
6           7      Bui Tien Dung   Kinh Te       34.0    9.2
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2
9          10       Cao Thi Hanh  May Tinh        NaN    7.0
   id course_name
0  12   Giai tich
1  34    Thong ke
2  26     Tin hoc
3  20      Dai so
4  24   Lap trinh


In [58]:
import sqlite3
import pandas as pd
import numpy as np

# Kết nối đến SQLite (cơ sở dữ liệu trong bộ nhớ)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng students
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Tạo bảng courses
cursor.execute("""
CREATE TABLE courses (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Chèn dữ liệu vào bảng students
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0),
]
cursor.executemany("INSERT INTO students VALUES (?, ?, ?, ?, ?);", student_data)

# Chèn dữ liệu vào bảng courses
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc"),
    (20, "Dai so"),
    (24, "Lap trinh"),
]
cursor.executemany("INSERT INTO courses VALUES (?, ?);", course_data)

#  Điền course_id bị thiếu với một giá trị hợp lệ
cursor.execute("""
UPDATE students
SET course_id = (SELECT id FROM courses ORDER BY RANDOM() LIMIT 1)
WHERE course_id IS NULL;
""")

#  Xóa bản ghi có course_id không tồn tại trong bảng courses
cursor.execute("""
DELETE FROM students
WHERE course_id NOT IN (SELECT id FROM courses);
""")

#  1️ Tích Descartes 
query = "SELECT * FROM students, courses;"
df_Tích_Descartes = pd.read_sql_query(query, conn)
print("\n Tích Descartes:\n", df_Tích_Descartes)

#  2️ INNER JOIN
query = """
SELECT students.*, courses.course_name 
FROM students 
INNER JOIN courses ON students.course_id = courses.id;
"""
df_inner_join = pd.read_sql_query(query, conn)
print("\n INNER JOIN:\n", df_inner_join)

# 3 LEFT JOIN
query = """
SELECT students.*, courses.course_name 
FROM students 
LEFT JOIN courses ON students.course_id = courses.id;
"""
df_left_join = pd.read_sql_query(query, conn)
print("\n LEFT JOIN:\n", df_left_join)

#  4️ RIGHT JOIN 
query = """
SELECT students.*, courses.course_name 
FROM courses
LEFT JOIN students ON students.course_id = courses.id;
"""
df_right_join = pd.read_sql_query(query, conn)
print("\n RIGHT JOIN :\n", df_right_join)

#  5️ FULL OUTER JOIN 
query = """
SELECT students.*, courses.course_name 
FROM students 
LEFT JOIN courses ON students.course_id = courses.id
UNION
SELECT students.*, courses.course_name 
FROM students 
RIGHT JOIN courses ON students.course_id = courses.id;
"""
df_full_outer_join = pd.read_sql_query(query, conn)
print("\n FULL OUTER JOIN :\n", df_full_outer_join)



# Đóng kết nối SQLite
conn.close()



 Tích Descartes:
     student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh         12    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh         12    6.7  20      Dai so
2            1  Nguyen Minh Hoang  May Tinh         12    6.7  24   Lap trinh
3            1  Nguyen Minh Hoang  May Tinh         12    6.7  26     Tin hoc
4            1  Nguyen Minh Hoang  May Tinh         12    6.7  34    Thong ke
5            2       Tran Thi Lan   Kinh Te         34    9.2  12   Giai tich
6            2       Tran Thi Lan   Kinh Te         34    9.2  20      Dai so
7            2       Tran Thi Lan   Kinh Te         34    9.2  24   Lap trinh
8            2       Tran Thi Lan   Kinh Te         34    9.2  26     Tin hoc
9            2       Tran Thi Lan   Kinh Te         34    9.2  34    Thong ke
10           3       Pham Van Nam  Toan Tin         24    7.9  12   Giai tich
11           3       Pham Van Nam  Toan Tin  

In [64]:
 # b. Tổng số sinh viên, điểm trung bình theo từng môn học
query = """
SELECT course_name, COUNT(student_id) AS total_students, AVG(score) AS avg_score
FROM students
JOIN courses ON students.course_id = courses.id
GROUP BY course_name;
"""
df_course_summary = pd.read_sql_query(query, conn)
print("\n Tổng số sinh viên và điểm trung bình của từng môn học:\n", df_course_summary)


 Tổng số sinh viên và điểm trung bình của từng môn học:
   course_name  total_students  avg_score
0      Dai so               2       8.00
1   Giai tich               1       6.70
2   Lap trinh               5       7.12
3    Thong ke               2       9.20


In [66]:
#  a. Tổng số sinh viên, điểm trung bình theo từng lớp
query = """
SELECT class, COUNT(student_id) AS total_students, AVG(score) AS avg_score
FROM students
GROUP BY class;
"""
df_class_summary = pd.read_sql_query(query, conn)
print("\n Tổng số sinh viên và điểm trung bình của từng lớp:\n", df_class_summary)


 Tổng số sinh viên và điểm trung bình của từng lớp:
       class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               4   6.800000
2  Toan Tin               3   7.966667


In [70]:
#  c. Phân loại thi đua theo số điểm bằng CASE WHEN
query = """
SELECT course_name, COUNT(student_id) AS total_students, AVG(score) AS avg_score,
    CASE
        WHEN AVG(score) >= 9.0 THEN 'Xuất sắc'
        WHEN AVG(score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
        ELSE 'Kém'
    END AS performance
FROM students
JOIN courses ON students.course_id = courses.id
GROUP BY course_name;
"""
df_performance = pd.read_sql_query(query, conn)
print("\n Phân loại thi đua theo số điểm:\n", df_performance)



 Phân loại thi đua theo số điểm:
   course_name  total_students  avg_score performance
0      Dai so               2       8.00         Tốt
1   Giai tich               1       6.70         Tốt
2   Lap trinh               5       7.12         Tốt
3    Thong ke               2       9.20    Xuất sắc


In [72]:
# 3 : Xếp hạng sinh viên theo điểm số toàn bộ
query = """
SELECT name, score, RANK() OVER (ORDER BY score DESC) AS rank
FROM students;
"""
df_ranked = pd.read_sql_query(query, conn)
print("\nXếp hạng sinh viên theo điểm số:\n", df_ranked)

# Xếp hạng sinh viên theo lớp
query = """
SELECT name, class, score, RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank
FROM students;
"""
df_ranked_class = pd.read_sql_query(query, conn)
print("\nXếp hạng sinh viên theo lớp:\n", df_ranked_class)

# Xếp hạng sinh viên theo môn học
query = """
SELECT name, course_id, score, RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank
FROM students;
"""
df_ranked_course = pd.read_sql_query(query, conn)
print("\nXếp hạng sinh viên theo môn học:\n", df_ranked_course)

# Top 3 sinh viên có điểm cao nhất
query = """
SELECT name, score FROM students
ORDER BY score DESC
LIMIT 3;
"""
df_top3 = pd.read_sql_query(query, conn)
print("\nTop 3 sinh viên đạt điểm cao nhất:\n", df_top3)

# Top 3 sinh viên có điểm thấp nhất
query = """
SELECT name, score FROM students
ORDER BY score ASC
LIMIT 3;
"""
df_bottom3 = pd.read_sql_query(query, conn)
print("\nTop 3 sinh viên đạt điểm thấp nhất:\n", df_bottom3)




Xếp hạng sinh viên theo điểm số:
                 name  score  rank
0       Tran Thi Lan    9.2     1
1      Bui Tien Dung    9.2     1
2        Ho Ngoc Mai    8.8     3
3        Vu Quoc Anh    8.0     4
4       Pham Van Nam    7.9     5
5     Le Thanh Huyen    7.2     6
6     Duong Huu Phuc    7.2     6
7       Cao Thi Hanh    7.0     8
8  Nguyen Minh Hoang    6.7     9
9     Dang Thuy Linh    5.5    10

Xếp hạng sinh viên theo lớp:
                 name     class  score  rank
0       Tran Thi Lan   Kinh Te    9.2     1
1      Bui Tien Dung   Kinh Te    9.2     1
2     Duong Huu Phuc   Kinh Te    7.2     3
3        Vu Quoc Anh  May Tinh    8.0     1
4       Cao Thi Hanh  May Tinh    7.0     2
5  Nguyen Minh Hoang  May Tinh    6.7     3
6     Dang Thuy Linh  May Tinh    5.5     4
7        Ho Ngoc Mai  Toan Tin    8.8     1
8       Pham Van Nam  Toan Tin    7.9     2
9     Le Thanh Huyen  Toan Tin    7.2     3

Xếp hạng sinh viên theo môn học:
                 name  course_id  score  r

In [85]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta

# Kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng students
cursor.execute("""
CREATE TABLE students (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL,
    graduation_date TEXT
);
""")

# Tạo bảng courses
cursor.execute("""
CREATE TABLE courses (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Chèn dữ liệu vào bảng students
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7, None),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2, None),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9, None),
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2, None),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0, None),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5, None),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2, None),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8, None),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2, None),
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0, None),
]
cursor.executemany("INSERT INTO students VALUES (?, ?, ?, ?, ?, ?);", student_data)

# Chèn dữ liệu vào bảng courses
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc"),
    (20, "Dai so"),
    (24, "Lap trinh"),
]
cursor.executemany("INSERT INTO courses VALUES (?, ?);", course_data)

# Cập nhật course_id bị thiếu
cursor.execute("""
UPDATE students
SET course_id = (SELECT id FROM courses ORDER BY RANDOM() LIMIT 1)
WHERE course_id IS NULL;
""")

# Loại bỏ bản ghi có course_id không tồn tại trong bảng courses
cursor.execute("""
DELETE FROM students
WHERE course_id NOT IN (SELECT id FROM courses);
""")

# Lấy danh sách sinh viên và tính ngày tốt nghiệp
students_df = pd.read_sql_query("SELECT * FROM students ORDER BY score DESC;", conn)
students_df["graduation_date"] = [(datetime.now() + timedelta(days=i+1)).strftime('%Y-%m-%d %H:%M:%S') for i in range(len(students_df))]

# Cập nhật giá trị graduation_date vào bảng
for index, row in students_df.iterrows():
    cursor.execute("""
    UPDATE students SET graduation_date = ? WHERE student_id = ?;
    """, (row["graduation_date"], row["student_id"]))



# Lấy thông tin ngày tốt nghiệp
query = """
SELECT name, score, graduation_date FROM students;
"""
df_graduation = pd.read_sql_query(query, conn)
print("\nThời gian tốt nghiệp của từng sinh viên:\n", df_graduation)

# Đóng kết nối SQLite
conn.close()


Thời gian tốt nghiệp của từng sinh viên:
                 name  score      graduation_date
0  Nguyen Minh Hoang    6.7  2025-03-18 10:27:06
1       Tran Thi Lan    9.2  2025-03-10 10:27:06
2       Pham Van Nam    7.9  2025-03-14 10:27:06
3     Le Thanh Huyen    7.2  2025-03-15 10:27:06
4        Vu Quoc Anh    8.0  2025-03-13 10:27:06
5     Dang Thuy Linh    5.5  2025-03-19 10:27:06
6      Bui Tien Dung    9.2  2025-03-11 10:27:06
7        Ho Ngoc Mai    8.8  2025-03-12 10:27:06
8     Duong Huu Phuc    7.2  2025-03-16 10:27:06
9       Cao Thi Hanh    7.0  2025-03-17 10:27:06
